In [1]:
import pandas as pd
import numpy as np
from keras.layers import Dense, Input, concatenate, Dropout, BatchNormalization, LSTM
from keras.models import Model, Sequential
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder,StandardScaler, Normalizer
import tensorflow as tf
import collections

Using TensorFlow backend.


In [10]:
dfTrain = pd.read_csv("training_set.csv")
dfMetaData = pd.read_csv("training_set_metadata.csv")
dfPoly = pd.read_csv("polynomial_coefficients_train.csv")
dfMerged = dfPoly.merge(dfMetaData,left_on = "object_id",right_on = "object_id")
dfMerged = dfMerged.drop(["distmod","object_id"],axis = 1)


In [11]:
num_hidden_neurons = 128
dropout = 0.25
epochs = 200


In [12]:
#preprocessing 
dfTrain, dfTest = train_test_split(dfMerged)

y_train = dfTrain["target"]
X_train = dfTrain.drop(["target"],axis = 1)

y_test = dfTest["target"]
X_test = dfTest.drop(["target"],axis = 1)

normalizer = Normalizer()
X_train = normalizer.fit_transform(X_train)
X_test = normalizer.transform(X_test)

#convert y to categorical 
unique_y = np.unique(y_train)
class_map = dict()
for i,val in enumerate(unique_y):
    class_map[val] = i
        
y_map = np.zeros((y_train.shape[0],))
y_map = np.array([class_map[val] for val in y_train])
y_categorical = to_categorical(y_map)

unique_y_test = np.unique(y_test)
class_map_test = dict()
for i,val in enumerate(unique_y_test):
    class_map_test[val] = i
        
y_map_test = np.zeros((y_test.shape[0],))
y_map_test = np.array([class_map_test[val] for val in y_test])
y_categorical_test = to_categorical(y_map_test)

In [13]:
#loss function
y_count = collections.Counter(y_map)
wtable = np.zeros((len(unique_y),))

for i in range(len(unique_y)):
    wtable[i] = y_count[i]/y_map.shape[0]
    
def mywloss(y_true,y_pred):  
    yc=tf.clip_by_value(y_pred,1e-15,1-1e-15)
    loss=-(tf.reduce_mean(tf.reduce_mean(y_true*tf.log(yc),axis=0)/wtable))
    return loss


In [14]:
model = Sequential()

model.add(Dense(num_hidden_neurons,input_dim = X_train[0].shape[0],activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(dropout))

model.add(Dense(14,activation = "softmax"))

model.compile(loss = mywloss,optimizer = "adam",metrics = ['acc'])

model.fit(X_train, y_categorical, batch_size = 128,validation_data=(X_test,y_categorical_test), epochs=epochs, verbose=1)

Train on 5886 samples, validate on 1962 samples
Epoch 1/200
5886/5886 [==============================] - 1s 138us/step - loss: 2.8436 - acc: 0.0919 - val_loss: 2.3356 - val_acc: 0.1789
Epoch 2/200
5886/5886 [==============================] - 0s 22us/step - loss: 2.4988 - acc: 0.1223 - val_loss: 2.2539 - val_acc: 0.1432
Epoch 3/200
5886/5886 [==============================] - 0s 22us/step - loss: 2.3798 - acc: 0.1359 - val_loss: 2.2026 - val_acc: 0.1498
Epoch 4/200
5886/5886 [==============================] - 0s 20us/step - loss: 2.3013 - acc: 0.1470 - val_loss: 2.1700 - val_acc: 0.1310
Epoch 5/200
5886/5886 [==============================] - 0s 22us/step - loss: 2.2456 - acc: 0.1544 - val_loss: 2.1428 - val_acc: 0.1315
Epoch 6/200
5886/5886 [==============================] - 0s 19us/step - loss: 2.2185 - acc: 0.1619 - val_loss: 2.1275 - val_acc: 0.1340
Epoch 7/200
5886/5886 [==============================] - 0s 23us/step - loss: 2.1889 - acc: 0.1507 - val_loss: 2.1121 - val_acc: 0.1417

5886/5886 [==============================] - 0s 32us/step - loss: 1.5520 - acc: 0.3349 - val_loss: 1.6274 - val_acc: 0.3660
Epoch 61/200
5886/5886 [==============================] - 0s 36us/step - loss: 1.5385 - acc: 0.3486 - val_loss: 1.6364 - val_acc: 0.3231
Epoch 62/200
5886/5886 [==============================] - 0s 22us/step - loss: 1.5326 - acc: 0.3478 - val_loss: 1.6094 - val_acc: 0.3542
Epoch 63/200
5886/5886 [==============================] - 0s 36us/step - loss: 1.5323 - acc: 0.3541 - val_loss: 1.5970 - val_acc: 0.3180
Epoch 64/200
5886/5886 [==============================] - 0s 37us/step - loss: 1.5224 - acc: 0.3379 - val_loss: 1.6088 - val_acc: 0.2880
Epoch 65/200
5886/5886 [==============================] - 0s 35us/step - loss: 1.5279 - acc: 0.3554 - val_loss: 1.6164 - val_acc: 0.3563
Epoch 66/200
5886/5886 [==============================] - 0s 36us/step - loss: 1.5068 - acc: 0.3556 - val_loss: 1.5798 - val_acc: 0.3578
Epoch 67/200
5886/5886 [==============================

Epoch 120/200
5886/5886 [==============================] - 0s 21us/step - loss: 1.2618 - acc: 0.4271 - val_loss: 1.4786 - val_acc: 0.4093
Epoch 121/200
5886/5886 [==============================] - 0s 20us/step - loss: 1.2781 - acc: 0.4225 - val_loss: 1.4550 - val_acc: 0.4292
Epoch 122/200
5886/5886 [==============================] - 0s 21us/step - loss: 1.2720 - acc: 0.4208 - val_loss: 1.4589 - val_acc: 0.4465
Epoch 123/200
5886/5886 [==============================] - 0s 21us/step - loss: 1.2634 - acc: 0.4229 - val_loss: 1.4218 - val_acc: 0.4587
Epoch 124/200
5886/5886 [==============================] - 0s 19us/step - loss: 1.2767 - acc: 0.4210 - val_loss: 1.4456 - val_acc: 0.4536
Epoch 125/200
5886/5886 [==============================] - 0s 19us/step - loss: 1.2645 - acc: 0.4280 - val_loss: 1.4600 - val_acc: 0.4490
Epoch 126/200
5886/5886 [==============================] - 0s 19us/step - loss: 1.2340 - acc: 0.4307 - val_loss: 1.4518 - val_acc: 0.4011
Epoch 127/200
5886/5886 [=========

5886/5886 [==============================] - 0s 19us/step - loss: 1.1745 - acc: 0.4400 - val_loss: 1.3870 - val_acc: 0.4169
Epoch 180/200
5886/5886 [==============================] - 0s 21us/step - loss: 1.1497 - acc: 0.4548 - val_loss: 1.4189 - val_acc: 0.4042
Epoch 181/200
5886/5886 [==============================] - 0s 20us/step - loss: 1.1629 - acc: 0.4619 - val_loss: 1.4280 - val_acc: 0.4230
Epoch 182/200
5886/5886 [==============================] - 0s 20us/step - loss: 1.1407 - acc: 0.4470 - val_loss: 1.4121 - val_acc: 0.4399
Epoch 183/200
5886/5886 [==============================] - 0s 19us/step - loss: 1.1644 - acc: 0.4501 - val_loss: 1.4410 - val_acc: 0.4786
Epoch 184/200
5886/5886 [==============================] - 0s 20us/step - loss: 1.1595 - acc: 0.4630 - val_loss: 1.3899 - val_acc: 0.4852
Epoch 185/200
5886/5886 [==============================] - 0s 20us/step - loss: 1.1499 - acc: 0.4585 - val_loss: 1.3875 - val_acc: 0.4725
Epoch 186/200
5886/5886 [=======================